In [2]:
import pickle
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorchtools
import glob as gl
import random
import os
import time
import itertools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import make_regression
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import log_loss
from numpy import hstack
from numpy import vstack

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()
find_directory(back_num = 1, foldername = ['Dataset'], filename = 'bat_dict.pkl')

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\All dataset\\Random seed test'

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\All dataset\\Dataset\\bat_dict.pkl'

In [4]:
bat_dict_add_1_2 = find_directory(back_num = 3, foldername = ['Dataset'], filename = 'bat_1_2_dict_VITQcQd.pkl')
with open(bat_dict_add_1_2, 'rb') as tf:
    bat_dict_1_2 = pickle.load(tf)

bat_sel_dict = bat_dict_1_2
max_time = 120

In [5]:
# batch1: 0~40(41)
# batch2: 41~83(43)
# batch3: 64~123(40)

b1 = 0
b2 = 0
b3 = 0

for cell in bat_sel_dict.keys():
    if cell.startswith('b1'):
        b1 += 1
    elif cell.startswith('b2'):
        b2 += 1
    else:
        b3 += 1

print(b1, b2, b3)

# Tensor dimension: (cell, cycle, time, variable)
regression_cycles = [30, 50, 80, 100]

# variable = [Qc, I, V, T, Qd]
num_variables = len(bat_sel_dict['b1c0']['1']['1.0'].keys())

for reg in regression_cycles:
    globals()[f'x_tensor_{reg}'] = torch.zeros(b1+b2+b3, reg, max_time, num_variables)
    cell_index = 1
    for cell in bat_sel_dict.keys():
        for cycle in range(1, reg+1):
            for time in bat_sel_dict[cell][str(cycle)].keys():
    #             print(bat_sel_dict[cell][str(cycle)][str(time)])
                value_list = [i for i in bat_sel_dict[cell][str(cycle)][str(time)].values()]
    #             print(value_list)
                globals()[f'x_tensor_{reg}'][cell_index-1, cycle-1, int(float(time)*max_time/60), :] = torch.FloatTensor(value_list)

        cell_index += 1
    
    globals()[f'x_tensor_{reg}'].size()
    
num_variables = globals()[f'x_tensor_{reg}'].shape[3]
time_length = globals()[f'x_tensor_{reg}'].shape[2]

41 43 40


torch.Size([124, 30, 120, 5])

torch.Size([124, 50, 120, 5])

torch.Size([124, 80, 120, 5])

torch.Size([124, 100, 120, 5])

In [6]:
y_df_add = find_directory(back_num = 3, foldername = ['Dataset', 'y_df.csv'])
y_df = pd.read_csv(y_df_add)

# 2. Regression problem
# KO, KP, CL
y_cl_numpy = y_df['Cycle life'].to_numpy()
y_kp_numpy = y_df['kneepoints'].to_numpy()
y_ko_numpy = y_df['kneeonsets'].to_numpy()
# y_ko_numpy.shape

y_regression_numpy = y_ko_numpy

y_ko_tensor = torch.FloatTensor(y_ko_numpy)
y_ko_tensor.size()

torch.Size([124])

In [7]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed) # torch 
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True # cudnn
    torch.backends.cudnn.benchmark = False # cudnn
    np.random.seed(random_seed) # numpy
    random.seed(random_seed) # random

In [8]:
globals()[f'x_tensor_{reg}'].shape

torch.Size([124, 100, 120, 5])

## Data loading

In [9]:
# y: no scaling
class dataPrep_RNN_CNN(Dataset):  
    def __init__(self, x_tensor, y_tensor, batch_size, scaler, batches, test_split: float, random_seed: int):
        self.random_seed = random_seed
        
        setRandomSeed(random_seed = self.random_seed)
        self.xdata = torch.permute(x_tensor, (0, 3, 2, 1))
        self.ydata = y_tensor
        
        self.batch_size = batch_size
                
        self.xscaler = scaler()
        batches = [0] + batches
        self.batches = batches
        self.test_split = test_split
        
        # Tensor dimension: (cell, variable, time, cycle)
        # If problem emerges, check this part first.
        # Issue: 각 batch마다 따로 scaling을 해야하나?
        for i in range(len(self.xdata)):
            for j in range(self.xdata.shape[1]):
                temp = np.expand_dims(self.xscaler.fit_transform(self.xdata[i,j,:,:]), axis=0)
                if j ==0:
                    temp2 = temp
                else:
                    temp2 = np.vstack((temp2,temp))
            temp2 = np.expand_dims(temp2, axis=0)
            if i==0:
                self.xdata_scaled = temp2
            else:
                self.xdata_scaled = np.vstack((self.xdata_scaled, temp2))

        self.ydata_scaled = self.ydata
        
#         self.all = list(zip(self.xdata_scaled, self.ydata_scaled))
        
        prev = 0
        print(f'random seed = {self.random_seed}')
        for i in range(1, len(self.batches)):
            globals()[f'self.b{i}_xdata_scaled'] = self.xdata_scaled[prev:prev + self.batches[i]]
            globals()[f'self.b{i}_ydata_scaled'] = self.ydata_scaled[prev:prev + self.batches[i]]
            
            prev +=self.batches[i]
            
            globals()[f'self.b{i}_train_size'], globals()[f'self.b{i}_val_size'], globals()[f'self.b{i}_test_size'] = self.get_splits_per_batch(self.batches[i])
            
            globals()[f'self.b{i}_x_train_scaled'], globals()[f'self.b{i}_x_test_scaled'], globals()[f'self.b{i}_y_train_scaled'], globals()[f'self.b{i}_y_test_scaled'] = train_test_split(globals()[f'self.b{i}_xdata_scaled'], 
                                                                                                            globals()[f'self.b{i}_ydata_scaled'], 
                                                                                                            test_size = globals()[f'self.b{i}_test_size'], 
                                                                                                            shuffle = True, 
                                                                                                            random_state=self.random_seed)
        
            globals()[f'self.b{i}_x_train_scaled'], globals()[f'self.b{i}_x_val_scaled'], globals()[f'self.b{i}_y_train_scaled'], globals()[f'self.b{i}_y_val_scaled'] = train_test_split(globals()[f'self.b{i}_x_train_scaled'], 
                                                                                                            globals()[f'self.b{i}_y_train_scaled'], 
                                                                                                            test_size = globals()[f'self.b{i}_val_size'], 
                                                                                                            shuffle = True, 
                                                                                                            random_state=self.random_seed)
            
            print(f'Batch {i}, total {self.batches[i]} cells')
            print('x_train, x_val, x_test:', globals()[f'self.b{i}_x_train_scaled'].shape, globals()[f'self.b{i}_x_val_scaled'].shape, globals()[f'self.b{i}_x_test_scaled'].shape)
            print('y_train, y_val, y_test:', globals()[f'self.b{i}_y_train_scaled'].shape, globals()[f'self.b{i}_y_val_scaled'].shape, globals()[f'self.b{i}_y_test_scaled'].shape)
        
            globals()[f'self.b{i}_train'] = list(zip(globals()[f'self.b{i}_x_train_scaled'], globals()[f'self.b{i}_y_train_scaled']))
            globals()[f'self.b{i}_val'] = list(zip(globals()[f'self.b{i}_x_val_scaled'], globals()[f'self.b{i}_y_val_scaled']))
            globals()[f'self.b{i}_test'] = list(zip(globals()[f'self.b{i}_x_test_scaled'], globals()[f'self.b{i}_y_test_scaled']))

            globals()[f'self.b{i}_train_dataloader'] = DataLoader(globals()[f'self.b{i}_train'], batch_size = self.batch_size, shuffle=True)
            globals()[f'self.b{i}_val_dataloader'] = DataLoader(globals()[f'self.b{i}_val'], batch_size = self.batch_size, shuffle=True)
            globals()[f'self.b{i}_test_dataloader'] = DataLoader(globals()[f'self.b{i}_test'], batch_size = self.batch_size, shuffle=True)
            
    # split the datasets into training and testing
    def get_splits_per_batch(self, batch_len):
        # test size
        test_size = int(self.test_split*batch_len)
        # val size
        val_size = int(0.2*(batch_len - test_size))
        # train_size
        train_size = batch_len - val_size - test_size
        return train_size, val_size, test_size
    
    # Returns dataloaders for training and validation datasets
    def train_data(self):
        self.train_dataloader = [globals()[f'self.b1_train_dataloader']]
        self.val_dataloader = [globals()[f'self.b1_val_dataloader']]
        if len(self.batches)>2:
            for i in range(2, len(self.batches)):
                self.train_dataloader.append(globals()[f'self.b{i}_train_dataloader'])
                self.val_dataloader.append(globals()[f'self.b{i}_val_dataloader'])
        return self.train_dataloader, self.val_dataloader
    
    # Returns dataloader for test datasets
    def test_data(self):
        self.test_dataloader = [globals()[f'self.b1_test_dataloader']]
        if len(self.batches)>2:
            for i in range(2, len(self.batches)):
                self.test_dataloader.append(globals()[f'self.b{i}_test_dataloader'])
        return self.test_dataloader
    
    def scaler(self):
        return self.xscaler
    
    # finding length of x 
    def __len__(self):
        return len(self.X)
    
    # indexing rows for calling
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]


In [10]:
regression_cycles = [30, 50, 80, 100]
batches = [b1, b2, b3]
random_seeds = [0, 50, 150, 200]

# x와 y가 1대1 대응이어야 dataloader를 만들 수 있어서 2D array 그대로 놓고 학습시 tensor transform 예정
for reg in regression_cycles:
    print(reg, 'cycles')
    for random_seed in random_seeds:
        globals()[f'reg_dataset_{reg}_{random_seed}'] = dataPrep_RNN_CNN(globals()[f'x_tensor_{reg}'], y_ko_tensor, batch_size = 4, scaler = MinMaxScaler, batches = batches, test_split = 0.2, random_seed = random_seed)
        globals()[f'reg_train_dataloader_{reg}_{random_seed}'], globals()[f'reg_val_dataloader_{reg}_{random_seed}'] = globals()[f'reg_dataset_{reg}_{random_seed}'].train_data()
        globals()[f'reg_test_dataloader_{reg}_{random_seed}'] = globals()[f'reg_dataset_{reg}_{random_seed}'].test_data()
        globals()[f'reg_xscaler_{reg}_{random_seed}'] = globals()[f'reg_dataset_{reg}_{random_seed}'].scaler()
        for batch in range(len(batches)):
            globals()[f'b{batch+1}_train_dataloader_{reg}_{random_seed}'], globals()[f'b{batch+1}_val_dataloader_{reg}_{random_seed}'], globals()[f'b{batch+1}_test_dataloader_{reg}_{random_seed}'] = globals()[f'reg_train_dataloader_{reg}_{random_seed}'][batch], globals()[f'reg_val_dataloader_{reg}_{random_seed}'][batch], globals()[f'reg_test_dataloader_{reg}_{random_seed}'][batch]

30 cycles
random seed = 0
Batch 1, total 41 cells
x_train, x_val, x_test: (27, 5, 120, 30) (6, 5, 120, 30) (8, 5, 120, 30)
y_train, y_val, y_test: torch.Size([27]) torch.Size([6]) torch.Size([8])
Batch 2, total 43 cells
x_train, x_val, x_test: (28, 5, 120, 30) (7, 5, 120, 30) (8, 5, 120, 30)
y_train, y_val, y_test: torch.Size([28]) torch.Size([7]) torch.Size([8])
Batch 3, total 40 cells
x_train, x_val, x_test: (26, 5, 120, 30) (6, 5, 120, 30) (8, 5, 120, 30)
y_train, y_val, y_test: torch.Size([26]) torch.Size([6]) torch.Size([8])
random seed = 50
Batch 1, total 41 cells
x_train, x_val, x_test: (27, 5, 120, 30) (6, 5, 120, 30) (8, 5, 120, 30)
y_train, y_val, y_test: torch.Size([27]) torch.Size([6]) torch.Size([8])
Batch 2, total 43 cells
x_train, x_val, x_test: (28, 5, 120, 30) (7, 5, 120, 30) (8, 5, 120, 30)
y_train, y_val, y_test: torch.Size([28]) torch.Size([7]) torch.Size([8])
Batch 3, total 40 cells
x_train, x_val, x_test: (26, 5, 120, 30) (6, 5, 120, 30) (8, 5, 120, 30)
y_train, y

## Define model class

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [12]:
class RNNcell(nn.Module):
    def __init__(self, model, input_size, hidden_size, dropout = 0.1, bidirectional = False, num_layer=1):
        super(RNNcell, self).__init__()
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
            
        self.model = model
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        
        self.D = 1 + self.bidirectional
        self.batch_size = 4
        
        #Batch size * cycles, timestep, input_size
        if self.model.endswith('RNN'):
            self.rnn = nn.RNN(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('LSTM'):
            self.rnn = nn.LSTM(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('GRU'):
            self.rnn = nn.GRU(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
            
        for name, param in self.rnn.named_parameters():
            if name.startswith('weight'):
                nn.init.xavier_uniform(param)
            else:
                nn.init.normal(param)
        
    def forward(self, x):
        # x made from (cell * n_cy, timestep(seq_len), num_vars)
        self.batch_size = x.size(0)
        
        D = 1 + self.bidirectional
        #out: Containing output features h_t from the last layer of LSTM for each t(if bidirectional ==True: concat of forward and backward hidden states)
        #h_n: Containing final hidden state in the sequence
        h0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
        if self.model.endswith('LSTM'):
            c0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
            # c_n: Containing final cell state in the sequence
            out, (hn, cn) = self.rnn(x, (h0, c0))
        else:
            out, hn = self.rnn(x, h0.detach())
        return out, hn

## Self attention for CA

In [13]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(SelfAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads
        
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, inputs):
        # inputs shape: [batch_size, seq_len, hidden_size]
        batch_size, seq_len, hidden_size = inputs.size()
        
        # Compute query, key, and value matrices
        query = self.query(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        key = self.key(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        value = self.value(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        
        # Compute scaled dot-product attention scores
        # attention_scores: [batch_size, num_heads, seq_len, seq_len]
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_size, dtype=torch.float32))
        attention_probs = self.softmax(attention_scores)
        
        # Apply attention to the value matrix and concatenate heads
        # context: [batch_size, num_heads, seq_len, head_size]=>[batch_size, seq_len, num_heads, head_size] 
        # => [batch_size, seq_len, num_heads * head_size]
        context = torch.matmul(attention_probs, value).transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
#         context = nn.Linear(hidden_size, 1)(context)
        return attention_probs, context

## RNN-TA-CA-1D CNN

In [15]:
class RNN_TA_CA_1DCNN(nn.Module):
    def __init__(self, input_size, num_time, num_cycles, rnn1, bi1, hid1, nhead2, fil2, pool2, npool2, fsize2, psize2, mids, random_seed, dr1 = 0.1, dr2 = 0.1, di2 = 1, st2 = 1, pad2 = 0):
        super(RNN_TA_CA_1DCNN, self).__init__()
        
        self.random_seed = random_seed
        setRandomSeed(random_seed = self.random_seed)
        
        self.D1 = 1 + bi1
        
        self.rnn1 = RNNcell(rnn1, input_size, hid1, dr1, bi1)
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        # Overall parameters
        self.num_time = num_time
        self.num_cycles = num_cycles
        self.mids = mids
        
        # Pooling/Nonpooling parameters
        # Pooling/Nonpooling layers
        self.pool2 = pool2
        self.npool2 = npool2
        # Pooling size(Pooling layer)
        self.psize2 = psize2
        
        # Params for self-attention(ca)
        self.nh2 = nhead2
        
        # CNN hyperparameters
        self.fil2 = fil2
        self.fsize2 = fsize2
        self.di2 = di2
        self.st2 = st2
        # Padding for CNN
        self.pad2 = pad2
                
        # Critical values for the 1st CNN layer
        # Batch for different cells
        
        # Conv1d takes input dimension: [Batch_size, c_in(no. of in_channels )= hidden_size, l_in = num_cycles]
        self.in2 = self.D1*hid1
        self.out2 = self.fil2
        self.Lout = self.num_cycles
        
        # Conv_block1~P
        # Pooling layers
        for i in range(1, self.pool2+1):
#             print(f"i: {i}, out_channels: {self.out_channels}")
            globals()[f'self.conv_block{i}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            nn.MaxPool1d(self.psize2)
            ).to(self.device)

            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int(int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)/self.psize2)

        for j in range(self.pool2+1, self.pool2+self.npool2+1):
            globals()[f'self.conv_block{j}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            ).to(self.device)
            
            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)
        
        self.lin1 = nn.Linear(self.D1*hid1, 1)
        self.sm1 = nn.Softmax(dim = 2)
        self.relu = nn.ReLU()
        
        self.ca = SelfAttention(self.D1*hid1, self.nh2)
        
        self.fc = nn.Sequential(nn.Linear(self.in2*self.Lout, self.mids[0], bias=True), nn.ReLU(),
                       nn.Linear(self.mids[0], self.mids[1], bias = True), nn.ReLU(),
                       nn.Linear(self.mids[1], self.mids[2], bias = True))
        
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        self.m = m
        if type(self.m) == nn.Linear:
            nn.init.xavier_uniform_(self.m.weight)
        
    def forward(self, x):
        # Original: Batch size, num_vars, timestep, cycles=>Batch size * cycles, timestep, input_size
        x = torch.reshape(x, (x.size(0)*x.size(3), x.size(2), x.size(1))).to(device)
#         print("Batch size * cycles, timestep, input_size: ", x.size())
        out1, hn1 = self.rnn1(x)
        
        # hn의 batch size가 n_cy*k로 설정
        cell_batch_size = int(hn1.size(1)/self.num_cycles)

        # Reshape for input
        # out1: (batch_size = num_cell*num_cycle, timestep, D1*hid1)
        # outs : (batch_size(num_cell), num_cycle, timestep, D1*hid1)
        outs1 = torch.reshape(out1, (cell_batch_size, -1, out1.size(1), out1.size(2)))
        # ta: (batch_size, num_cycle, timestep, 1(D1*hid1->1))
        ta = self.sm1(self.lin1(outs1))
        # ta_outs: (batch_size, num_cycle, timestep, D1*hid1)=>(batch_size, D1*hid1, num_cycle, timestep)
        ta_outs = torch.permute(ta*outs1, (0, 3, 1, 2))
        # ct_vec: (batch_size, D1*hid1, num_cycle)
        ct_vec = torch.sum(ta_outs, -1)
        # ct_vec shape: [batch_size, num_cycle, D1*hid1(hidden_size)]
        # hidden_size = num_heads * head_size
        # ct_vec transform back: [batch_size, num_heads, num_cycle, head_size]=>[batch_size, num_cycle, num_heads, head_size] 
        # => [batch_size, num_cycle, num_heads * head_size(hidden_size)]
        # ca: matmul(query[batch_size, num_heads, num_cycle, head_size], key^T[batch_size, num_heads, head_size, num_cycle])
        # ca shape: [batch_size, num_heads, num_cycle, num_cycle]
        ca, ct_vec = self.ca(ct_vec.transpose(2, 1))
#         print(f"After self attention: ca.size: {ca.size()}, ct_vec.size: {ct_vec.size()}")
        # ct_vec shape: [batch_size, D1*hid1, num_cycle]
        ct_vec = ct_vec.transpose(2, 1)
        
        
        # input for 1d cnn: (batch_size, D1*hid1, num_cycle)
        for i in range(1, self.pool2+self.npool2+1):
            ct_vec = globals()[f'self.conv_block{i}'](ct_vec)
        # ct_vec from final convolutional layer: (batch_size, self.in2*self.Lout)
        ct_vec = ct_vec.view(ct_vec.size(0), -1)
        # final_out: (batch_size, 1)
        final_out = self.fc(ct_vec)
        
        return final_out.squeeze(), ta.squeeze(), ca

In [16]:
# Function for saving and loading of training history
def save_data(D3_array, filename):
    with open(filename,"wb") as dat_:
        pickle.dump(D3_array,dat_)
        
def load_data(filename):
    with open(filename,"rb") as ld:
        x_temp = pickle.load(ld)
    return x_temp

# Early stopping class
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.best_epoch = 1
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, epoch, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model = self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_epoch = epoch
            self.best_model = self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'At epoch {self.best_epoch}: Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
        return model

In [17]:
class train:
    def __init__(self, modelname, model: nn.Module, train_dataloader, val_dataloader, test_dataloader, epoch: int, learning_rate=0.01, patience = 5, random_seed = 100, verbose = False):
        super().__init__()
        #   Reprocudtion
        self.random_seed = random_seed
        self.modelname = modelname
        setRandomSeed(self.random_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(self.random_seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        os.environ['PYTHONHASHSEED'] = str(self.random_seed)
        torch.manual_seed(self.random_seed)
        np.random.seed(self.random_seed)

        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.test_dataloader = test_dataloader
        self.epoch = epoch
        self.learning_rate = learning_rate
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.learning_rate)
        self.early_stopping = EarlyStopping(patience = patience, verbose = verbose)

        self.criterion = nn.MSELoss()
        self.name = ['train_rmse', 'val_rmse', 'test_rmse']
        
    def EvalModel(self):
        EvalPredictions, EvalActuals = list(), list()
        TestPredictions, TestActuals = list(), list()
        self.model.eval()
        with torch.no_grad():
            for n, (inputs, targets) in enumerate(self.val_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)
                
                if self.modelname == 'RNN_TA_1DCNN':
                    yhat, ta = self.model(inputs)
                elif self.modelname == 'RNN_CA_1DCNN':
                    yhat, ca = self.model(inputs)
                else:
                    yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                if n==0:
                    EvalPredictions = yhat
                    EvalActuals = actual
                else:
                    EvalPredictions = np.concatenate((EvalPredictions, yhat), axis = 0)
                    EvalActuals = np.concatenate((EvalActuals, actual), axis = 0)
                    
            val_rmse = mean_squared_error(EvalActuals, EvalPredictions, squared = False)
            
            for n, (inputs, targets) in enumerate(self.test_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                if self.modelname == 'RNN_TA_1DCNN':
                    yhat, ta = self.model(inputs)
                elif self.modelname == 'RNN_CA_1DCNN':
                    yhat, ca = self.model(inputs)
                else:
                    yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                if n==0:
                    TestPredictions = yhat
                    TestActuals = actual
                else:
                    TestPredictions = np.concatenate((TestPredictions, yhat), axis = 0)
                    TestActuals = np.concatenate((TestActuals, actual), axis = 0)
                    
            test_rmse = mean_squared_error(TestActuals, TestPredictions, squared = False)
            
            return val_rmse, test_rmse

    def TrainModel(self):
        loss_history = []
        self.model.train()
        for i in range(self.epoch):
            predictions, actuals = list(), list()
            for n, (inputs, targets) in enumerate(self.train_dataloader):
                self.model.train()
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                self.optimizer.zero_grad()

                if self.modelname == 'RNN_TA_1DCNN':
                    yhat, ta = self.model(inputs)
                elif self.modelname == 'RNN_CA_1DCNN':
                    yhat, ca = self.model(inputs)
                else:
                    yhat, ta, ca = self.model(inputs)
                    
                loss = self.criterion(yhat, targets)
                loss.backward()
                self.optimizer.step()
                
                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')
                    
                yhat = yhat.detach().numpy()
                targets = targets.numpy()
                
                if n==0:
                    predictions = yhat
                    actuals = targets
                else:
                    predictions = np.concatenate((predictions, yhat), axis = 0)
                    actuals = np.concatenate((actuals, targets), axis = 0)

            train_rmse = mean_squared_error(actuals, predictions, squared = False)

            # evaluation on validation data
            val_rmse, test_rmse = self.EvalModel()
            early_factor = val_rmse
            
            # training loss history
            loss_history.append([train_rmse, val_rmse, test_rmse])

            self.early_stopping(i+1, early_factor, self.model)
            
            if self.early_stopping.early_stop:
                print("Early stopping at best epoch: ", self.early_stopping.best_epoch)
                self.best_loss = loss_history[self.early_stopping.best_epoch-1]
                loss_history = loss_history[:self.early_stopping.best_epoch]
                
                if self.modelname == 'RNN_TA_1DCNN':
                    return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta
                elif self.modelname == 'RNN_CA_1DCNN':
                    return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ca
                else:
                    return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
            
        print(f"Ends at final epoch {self.epoch}")
        print(f"Best epoch: {self.early_stopping.best_epoch}")
        self.best_loss = loss_history[self.early_stopping.best_epoch-1]
        loss_history = loss_history[:self.early_stopping.best_epoch]
        
        if self.modelname == 'RNN_TA_1DCNN':
            return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta
        elif self.modelname == 'RNN_CA_1DCNN':
            return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ca
        else:
            return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
        
    def predict(self, data: torch.Tensor):
        return self.model(data)

## Implementation

In [18]:
def noheads_history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr, batch_num, random_seed):
    history_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num+1}', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.pkl')
    state_dict_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num+1}', 'model'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_state_dict.pth')
    ta_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num+1}', 'ta'],
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    return history_add, state_dict_add, ta_add

def multihead_history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, batch_num, random_seed):
    history_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num+1}', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.pkl')
    state_dict_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num+1}', 'model'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_state_dict.pth')
    ta_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num+1}', 'ta'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    ca_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Random_seed = {random_seed}', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num+1}', 'ca'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    return history_add, state_dict_add, ta_add, ca_add

## RNN-TA-CA-1D CNN, random seed 다르게
## 각 batch 당 결과 보고 batch 특성 고려해서 더 input 줄일수 있다는 결과 필요(새로운 결과)

In [ ]:
# Hyperparameters
num_vars = globals()[f'x_tensor_{reg}'].size()[3]

modelnames = ['RNN_TA_CA_1DCNN']
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycless = [100, 30, 50, 80]
# random_seeds = [0, 50, 150, 200, 100]
random_seeds = [150, 200, 100]
ep_pats = [[2000, 20], [3000, 30]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [1, 2, 3, 4, 5]

batch_nums = [2, 1, 0]

import time

for random_seed in random_seeds:
    for batch_num in batch_nums:
        for modelname in modelnames:
            for [n_ep, patience] in ep_pats:
                for num_cycles in num_cycless:
                    df = pd.DataFrame(columns = ['train_rmse', 'val_rmse', 'test_rmse'])
                    print(f"random_seed = {random_seed}, modelname: {modelname}, Batch: {batch_num + 1}, num_cycle = {num_cycles}, epoch = {n_ep}, patience = {patience}")
                    trdl = globals()[f'b{batch_num+1}_train_dataloader_{num_cycles}_{random_seed}']
                    vdl = globals()[f'b{batch_num+1}_val_dataloader_{num_cycles}_{random_seed}']
                    tedl = globals()[f'b{batch_num+1}_test_dataloader_{num_cycles}_{random_seed}']
                    for rnn1, hid1 in itertools.product(rnns, hids):
                        bi1 = False
                        for pool2, npool2 in itertools.product(pools, npools):
                            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers')
                            for fil2, lr in itertools.product(num_fils, lrs):
                                if modelname == 'RNN_TA_1DCNN':
                                    history_add, state_dict_add, ta_add = noheads_history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                            rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr, batch_num, random_seed)
                                    if os.path.isfile(history_add):
                                        history = load_data(history_add)
                                        df.loc[f'{rnn1}_{hid1}_{pool2}_{npool2}_{fil2}_{lr}'] = history.iloc[-1, :]
                                    else:
                                        print(f"rnn = {rnn1}, hid = {hid1}, num_fil = {fil2}, lr = {lr}")
                                        start = time.time()
                                        model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, hid1, fil2, pool2, npool2, fsize2, psize2, mids, random_seed).to(device)
                                        model_train = train(modelname, model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False, random_seed = random_seed)
                                        best_model, best_loss, history, ta = model_train.TrainModel()

                                        print('best_loss = '+ str(best_loss))
                                        save_data(ta, ta_add)
                                        save_data(history, history_add)
                                        save_data(best_model.state_dict(), state_dict_add)

                                        print("time: ", time.time()-start)
                                        print("\n\n")
                                        df.loc[f'{rnn1}_{hid1}_{pool2}_{npool2}_{fil2}_{lr}'] = history.iloc[-1, :]
                                else:
                                    for nh2 in nh2s:
                                        
                                        all_hid1 = hid1 * nh2

                                        # Train
                                        
                                        history_add, state_dict_add, ta_add, ca_add = multihead_history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                            rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, batch_num, random_seed)
                                        if os.path.isfile(history_add):
                                            history = load_data(history_add)
                                            df.loc[f'{rnn1}_{hid1}_{pool2}_{npool2}_{fil2}_{lr}_{nh2}'] = history.iloc[-1, :]
                                            
                                        else:
                                            print(f"rnn = {rnn1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")
                                            start = time.time()
                                            model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids, random_seed).to(device)
                                            model_train = train(modelname, model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False, random_seed = random_seed)
                                            if modelname == 'RNN_CA_1DCNN':
                                                best_model, best_loss, history, ca = model_train.TrainModel()
                                            else:
                                                best_model, best_loss, history, ta, ca = model_train.TrainModel()
                                                save_data(ta, ta_add)

                                            print('best_loss = '+ str(best_loss))
                                            save_data(history, history_add)
                                            save_data(best_model.state_dict(), state_dict_add)
                                            save_data(ca, ca_add)

                                            print("time: ", time.time()-start)
                                            print("\n\n")
                                            df.loc[f'{rnn1}_{hid1}_{pool2}_{npool2}_{fil2}_{lr}_{nh2}'] = history.iloc[-1, :]
                                            
                    dfadd = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', 'rstest_results', modelname], 
                        filename = f'{n_ep}_{patience}_rs_{random_seed}_split_1_2_{modelname}_batch_{batch_num}_{num_cycles} cycles.csv')
                    df.to_csv(dfadd)                                                                                                                                                                                       

random_seed = 150, modelname: RNN_TA_CA_1DCNN, Batch: 3, num_cycle = 100, epoch = 2000, patience = 20
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers,

Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers


Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
random_seed = 150, modelname: RNN_TA_CA_1DCNN, Batch: 2, num_cycle = 80, epoch = 2000, patience = 20
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, 

Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers


Early stopping at best epoch:  422
best_loss = [360.94745, 199.55147, 535.14856]
time:  25.219144582748413



rnn = LSTM, hid per head = 3, num_heads = 3, num_fil = 7, lr = 0.0001
Ends at final epoch 2000
Best epoch: 2000
best_loss = [601.7888, 456.84808, 807.22266]
time:  115.87466645240784



rnn = LSTM, hid per head = 3, num_heads = 4, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  359
best_loss = [366.1043, 210.42636, 591.0506]
time:  21.04715371131897



rnn = LSTM, hid per head = 3, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  422
best_loss = [300.64325, 141.40239, 460.73938]
time:  24.562487602233887



rnn = LSTM, hid per head = 3, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  64
best_loss = [282.39523, 70.48225, 492.46973]
time:  4.3285815715789795



rnn = LSTM, hid per head = 3, num_heads = 2, num_fil = 7, lr = 0.001
Early stopping at best epoch:  59
best_loss = [199.92773, 56.262268, 356.52136]
time:  4.372595071792

...

Early stopping at best epoch:  85
best_loss = [188.5036, 85.32999, 403.26123]
time:  6.034315586090088



rnn = LSTM, hid per head = 5, num_heads = 4, num_fil = 3, lr = 0.001
Early stopping at best epoch:  100
best_loss = [171.81567, 60.35905, 370.9747]
time:  6.765416145324707



rnn = LSTM, hid per head = 5, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  113
best_loss = [217.55202, 75.00197, 316.16428]
time:  7.483978509902954



rnn = LSTM, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  42
best_loss = [98.64169, 85.33567, 411.48285]
time:  3.4689853191375732



rnn = LSTM, hid per head = 5, num_heads = 2, num_fil = 3, lr = 0.01
Early stopping at best epoch:  11
best_loss = [305.31802, 77.57531, 458.14737]
time:  2.20332670211792



rnn = LSTM, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  45
best_loss = [86.81495, 88.96396, 378.36435]
time:  3.875073194503784



rnn = LSTM, hid pe

Early stopping at best epoch:  103
best_loss = [215.23488, 40.95888, 437.78845]
time:  7.422217845916748



rnn = LSTM, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  9
best_loss = [300.03073, 104.02975, 376.30902]
time:  1.9997386932373047



rnn = LSTM, hid per head = 5, num_heads = 2, num_fil = 3, lr = 0.01
Early stopping at best epoch:  81
best_loss = [85.675934, 61.438812, 424.81192]
time:  6.171993255615234



rnn = LSTM, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  17
best_loss = [263.4553, 66.907425, 479.57343]
time:  2.578096628189087



rnn = LSTM, hid per head = 5, num_heads = 4, num_fil = 3, lr = 0.01
Early stopping at best epoch:  42
best_loss = [74.21279, 59.757885, 408.67075]
time:  3.875291585922241



rnn = LSTM, hid per head = 5, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  89
best_loss = [57.433327, 22.434774, 339.90158]
time:  6.452274322509766



rnn = LSTM, hi

Early stopping at best epoch:  61
best_loss = [123.28615, 75.223404, 426.13422]
time:  4.812578916549683



rnn = LSTM, hid per head = 5, num_heads = 2, num_fil = 3, lr = 0.01
Early stopping at best epoch:  32
best_loss = [146.14293, 82.64175, 445.7439]
time:  3.452483654022217



rnn = LSTM, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  80
best_loss = [69.641655, 59.301758, 395.0411]
time:  6.031173229217529



rnn = LSTM, hid per head = 5, num_heads = 4, num_fil = 3, lr = 0.01
Ends at final epoch 3000
Best epoch: 3000
best_loss = [432.10516, 262.25067, 645.5832]
time:  169.42245364189148



rnn = LSTM, hid per head = 5, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  18
best_loss = [239.2982, 80.46231, 390.65164]
time:  2.640050172805786



rnn = LSTM, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1152
best_loss = [261.39148, 72.20614, 497.46777]
time:  64.25058579444885



rnn = 

Early stopping at best epoch:  13
best_loss = [301.70844, 59.256485, 530.4045]
time:  2.4532933235168457



rnn = LSTM, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  29
best_loss = [266.66428, 52.061493, 465.26114]
time:  3.3431308269500732



rnn = LSTM, hid per head = 5, num_heads = 4, num_fil = 3, lr = 0.01
Ends at final epoch 3000
Best epoch: 3000
best_loss = [432.09756, 262.24142, 645.5759]
time:  172.54743838310242



rnn = LSTM, hid per head = 5, num_heads = 5, num_fil = 3, lr = 0.01
Ends at final epoch 3000
Best epoch: 3000
best_loss = [432.4471, 262.6674, 645.9115]
time:  171.76544332504272



rnn = LSTM, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  944
best_loss = [379.4732, 213.703, 608.227]
time:  54.891108989715576



rnn = LSTM, hid per head = 5, num_heads = 2, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1207
best_loss = [271.96863, 88.03594, 489.07492]
time:  69.6715853214263

Early stopping at best epoch:  11
best_loss = [239.20732, 82.08891, 384.70825]
time:  2.1562769412994385



rnn = LSTM, hid per head = 7, num_heads = 4, num_fil = 3, lr = 0.01
Early stopping at best epoch:  52
best_loss = [88.2751, 44.872864, 408.51187]
time:  4.203730344772339



rnn = LSTM, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.01
Ends at final epoch 3000
Best epoch: 3000
best_loss = [432.2244, 262.39597, 645.69763]
time:  160.7971215248108



rnn = LSTM, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  820
best_loss = [216.19817, 99.85761, 461.02325]
time:  42.770002126693726



rnn = LSTM, hid per head = 7, num_heads = 2, num_fil = 5, lr = 0.0001
Ends at final epoch 3000
Best epoch: 3000
best_loss = [600.8796, 455.84872, 806.35706]
time:  152.2604968547821



rnn = LSTM, hid per head = 7, num_heads = 3, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  980
best_loss = [264.9638, 93.07784, 449.16183]
time:  51.156327247619

In [ ]:
# Hyperparameters
num_vars = globals()[f'x_tensor_{reg}'].size()[3]

modelnames = ['RNN_TA_CA_1DCNN']
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycless = [100, 30, 50, 80]
random_seeds = [0, 50, 150, 200, 100]
ep_pats = [[2000, 20], [3000, 30]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [1, 2, 3, 4, 5]

batch_nums = [2, 1, 0]

import time

for random_seed in random_seeds:
    for batch_num in batch_nums:
        for modelname in modelnames:
            for [n_ep, patience] in ep_pats:
                for num_cycles in num_cycless:
                    df = pd.DataFrame(columns = ['train rmse', 'val rmse', 'test rmse'])
                    
                    print(f"random_seed = {random_seed}, Batch: {batch_num + 1}, num_cycle = {num_cycles}, epoch = {n_ep}, patience = {patience}")
                    trdl = globals()[f'b{batch_num+1}_train_dataloader_{num_cycles}_{random_seed}']
                    vdl = globals()[f'b{batch_num+1}_val_dataloader_{num_cycles}_{random_seed}']
                    tedl = globals()[f'b{batch_num+1}_test_dataloader_{num_cycles}_{random_seed}']
                    for rnn1, hid1 in itertools.product(rnns, hids):
                        bi1 = False
                        for pool2, npool2 in itertools.product(pools, npools):
                            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers')
                            for fil2, lr in itertools.product(num_fils, lrs):
                                if modelname == 'RNN_TA_1DCNN':
                                    
                                    print(f"rnn = {rnn1}, hid = {hid1}, num_fil = {fil2}, lr = {lr}")
                                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, hid1, fil2, pool2, npool2, fsize2, psize2, mids, random_seed).to(device)
                                    history_add, state_dict_add, ta_add = noheads_history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                            rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr, batch_num, random_seed)
                                    if os.path.isfile(history_add):
                                        df.loc[f'{rnn1}_{hid1}_{pool2}_{npool2}_{fil2}_{lr}'] = load_data(history_add)
                                    else:
                                        start = time.time()
                                        model_train = train(modelname, model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False, random_seed = random_seed)
                                        best_model, best_loss, history, ta = model_train.TrainModel()

                                        print('best_loss = '+ str(best_loss))
                                        save_data(ta, ta_add)
                                        save_data(history, history_add)
                                        save_data(best_model.state_dict(), state_dict_add)

                                        print("time: ", time.time()-start)
                                        print("\n\n")
                                else:
                                    for nh2 in nh2s:
                                        print(f"rnn = {rnn1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")
                                        all_hid1 = hid1 * nh2
                                        model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids, random_seed).to(device)

                                        # Train
                                        
                                        history_add, state_dict_add, ta_add, ca_add = multihead_history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                            rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, batch_num, random_seed)
                                        if os.path.isfile(history_add):
                                            df.loc[f'{rnn1}_{hid1}_{pool2}_{npool2}_{fil2}_{lr}_{nh1}'] = load_data(history_add)
                                        else:
                                            start = time.time()
                                            model_train = train(modelname, model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False, random_seed = random_seed)
                                            if modelname == 'RNN_CA_1DCNN':
                                                best_model, best_loss, history, ca = model_train.TrainModel()
                                            else:
                                                best_model, best_loss, history, ta, ca = model_train.TrainModel()
                                                save_data(ta, ta_add)

                                            print('best_loss = '+ str(best_loss))
                                            save_data(history, history_add)
                                            save_data(best_model.state_dict(), state_dict_add)
                                            save_data(ca, ca_add)

                                            print("time: ", time.time()-start)
                                            print("\n\n")